In [ ]:
from utils.fixations import (
    fixation_durations_and_final_clicks,
    compute_fixation_metrics_collapsed,
    compute_fixation_metrics_per_rating
)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

base_dir = r"eye-data-sept2025\EXPERIMENTS_selective"

In [9]:
fixation_durations_and_final_clicks(base_dir)

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\utils\fixations.py:108: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))



Stimulus category: POSITIVE
Total number of rating trials processed: 500
Top 1 longest fixation duration, with matching button click: 214
Top 2 longest fixation duration, with matching button click: 88
Top 3 longest fixation duration, with matching button click: 48
Top 4 longest fixation duration, with matching button click: 23
Top 5 longest fixation duration, with matching button click: 5
Incorrect (doesn't match any top 5 fixation durations): 122

Stimulus category: NEUTRAL
Total number of rating trials processed: 500
Top 1 longest fixation duration, with matching button click: 186
Top 2 longest fixation duration, with matching button click: 98
Top 3 longest fixation duration, with matching button click: 39
Top 4 longest fixation duration, with matching button click: 18
Top 5 longest fixation duration, with matching button click: 7
Incorrect (doesn't match any top 5 fixation durations): 152

Stimulus category: NEGATIVE
Total number of rating trials processed: 500
Top 1 longest fixat

In [10]:
subject_collapsed, group_collapsed = compute_fixation_metrics_collapsed(base_dir)

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\utils\fixations.py:280: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))


Saved subject-level metrics to: fixation_metrics_collapsed\subject_level_fixation_metrics_collapsed.csv
Saved group-level metrics to: fixation_metrics_collapsed\group_level_fixation_metrics_collapsed.csv


In [ ]:
subject_collapsed

: 

In [ ]:
group_collapsed

: 

In [11]:
subject_per_rating, group_per_rating = compute_fixation_metrics_per_rating(base_dir)

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\utils\fixations.py:355: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))


Saved subject-level metrics to: fixation_metrics_per_emotion\subject_level_fixation_metrics_per_emotion.csv
Saved group-level metrics to: fixation_metrics_per_emotion\group_level_fixation_metrics_per_emotion.csv


In [12]:
subject_per_rating

,subject_id,stim_id,emotion_category,emotion,fixation_count,mean_duration,total_duration
0,anjana,Animals_025_h.jpg,NA,disgust,1,19.168000,19.168
1,anjana,Animals_025_h.jpg,NA,sad,7,217.623286,1523.363
2,anjana,Animals_025_h.jpg,NA,happy,6,170.003833,1020.023
3,anjana,Animals_025_h.jpg,NA,anger,6,239.033167,1434.199
4,anjana,Animals_025_h.jpg,NA,fear,8,191.563500,1532.508
...,...,...,...,...,...,...,...
1495,vaishnavi,People_240_h.jpg,NA,anger,56,309.599821,17337.590
1496,vaishnavi,People_240_h.jpg,NA,sad,33,160.935545,5310.873
1497,vaishnavi,People_240_h.jpg,NA,fear,19,183.336158,3483.387
1498,vaishnavi,People_240_h.jpg,NA,disgust,15,235.947133,3539.207


In [13]:
group_per_rating

,emotion_category,emotion,mean_fixation_count,sd_fixation_count,mean_fixation_duration,sd_fixation_duration,mean_total_duration,sd_total_duration
0,NA,anger,6.640000,5.349060,210.184973,81.072146,1333.374533,1338.793413
1,NA,disgust,6.990000,4.843010,211.785425,94.077538,1388.222723,1029.408765
2,NA,fear,7.253333,6.366390,206.324372,74.664279,1385.277453,1272.601112
3,NA,happy,7.830000,6.554717,211.255903,90.396644,1541.523470,1277.907097
4,NA,sad,7.516667,5.641733,211.118069,89.531344,1454.030030,1033.446964


In [16]:
import pandas as pd
df = pd.read_csv(r"D:\GITHUB\eye-tracking-ccs\preprocessing & analysis\eye-data-sept2025\EXPERIMENTS_selective\deepali\eye_tracking_data.csv")
rating_rows = df[df['remarks'].notna() & df['remarks'].str.endswith("_EMOTION_RATING")]
print(rating_rows[['stim_cat', 'remarks']].head())

       stim_cat              remarks
14239  negative  fear_EMOTION_RATING
14240  negative  fear_EMOTION_RATING
14241  negative  fear_EMOTION_RATING
14242  negative  fear_EMOTION_RATING
14243  negative  fear_EMOTION_RATING


---

In [18]:
import os
import numpy as np
from collections import defaultdict
from pygazeanalyser.detectors import fixation_detection
from utils.common import extract_emotion_rating_segments, convert_to_pygaze_compatible_format

# emotion ratings grouped by stim_cat
def compute_fixation_metrics_collapsed_this(base_dir, save_dir="fixation_metrics_collapsed"):
    """
    Compute fixation metrics for emotion rating screens,
    collapsed within each emotion category (positive, neutral, negative).
    Produces both subject-level and group-level summaries.
    """

    subjects = os.listdir(base_dir)
    subjects = [os.path.join(base_dir, s) for s in subjects]

    subject_summaries = []

    for s in subjects:
        subject_id = os.path.splitext(os.path.basename(s))[0]
        df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))

        # --- CLEAN COLUMN NAMES AND STIM_CAT ---
        df.columns = df.columns.str.strip()
        df['stim_cat'] = df['stim_cat'].astype(str).str.strip().replace({'nan': np.nan})

        # Extract rating segments
        rating_segs = extract_emotion_rating_segments(df)

        # Container to accumulate by category
        category_accum = defaultdict(list)

        for stim_id, emotions_dict in rating_segs.items():
            for emotion, seg_df in emotions_dict.items():

                # --- DEBUG: check stim_cat values in each rating segment ---
                print("SUBJECT", subject_id, "STIM", stim_id, "EMOTION", emotion)
                print(seg_df[['stim_cat', 'stim_id', 'remarks']].head())
                print(
                    "raw value repr:",
                    repr(seg_df['stim_cat'].iloc[0]),
                    "pd.isna:",
                    pd.isna(seg_df['stim_cat'].iloc[0]),
                )

                # --- Assign emotion category safely ---
                cat = seg_df['stim_cat'].iloc[0]
                if pd.isna(cat) or str(cat).strip() == "":
                    # fallback: look up in full dataframe
                    fallback = df.loc[df['stim_id'] == stim_id, 'stim_cat']
                    if not fallback.empty:
                        cat = (
                            fallback.ffill()
                            .bfill()
                            .dropna()
                            .astype(str)
                            .str.strip()
                            .iloc[0]
                            if fallback.dropna().any()
                            else "UNKNOWN"
                        )
                    else:
                        cat = "UNKNOWN"
                else:
                    cat = str(cat).strip()

                emotion_category = cat

                # --- Calculate fixations ---
                x, y, t = convert_to_pygaze_compatible_format(seg_df)
                _, Efix = fixation_detection(x, y, t)

                durations = [f[2] for f in Efix] if Efix else []
                if durations:
                    category_accum[emotion_category].extend(durations)

        # Compute mean per category
        for category, dur_list in category_accum.items():
            fixation_count = len(dur_list)
            mean_duration = np.mean(dur_list)
            total_duration = np.sum(dur_list)

            subject_summaries.append(
                {
                    "subject_id": subject_id,
                    "emotion_category": category,
                    "fixation_count": fixation_count,
                    "mean_duration": mean_duration,
                    "total_duration": total_duration,
                }
            )

    # ---- Convert to DataFrame ----
    subject_df = pd.DataFrame(subject_summaries)

    # ---- Group-level summaries ----
    group_df = (
        subject_df.groupby("emotion_category")
        .agg(
            mean_fixation_count=("fixation_count", "mean"),
            sd_fixation_count=("fixation_count", "std"),
            mean_fixation_duration=("mean_duration", "mean"),
            sd_fixation_duration=("mean_duration", "std"),
            mean_total_duration=("total_duration", "mean"),
            sd_total_duration=("total_duration", "std"),
        )
        .reset_index()
    )

    # ---- Save results ----
    os.makedirs(save_dir, exist_ok=True)
    subject_path = os.path.join(save_dir, "subject_level_fixation_metrics_collapsed.csv")
    group_path = os.path.join(save_dir, "group_level_fixation_metrics_collapsed.csv")

    subject_df.to_csv(subject_path, index=False)
    group_df.to_csv(group_path, index=False)

    print(f"Saved subject-level metrics to: {subject_path}")
    print(f"Saved group-level metrics to: {group_path}")

    return subject_df, group_df


subject_df, group_df = compute_fixation_metrics_collapsed_this(base_dir)

SUBJECT anjana STIM Animals_025_h.jpg EMOTION disgust
        stim_cat            stim_id                 remarks
450117  negative  Animals_025_h.jpg  disgust_EMOTION_RATING
450118  negative  Animals_025_h.jpg  disgust_EMOTION_RATING
450119  negative  Animals_025_h.jpg  disgust_EMOTION_RATING
450120  negative  Animals_025_h.jpg  disgust_EMOTION_RATING
450121  negative  Animals_025_h.jpg  disgust_EMOTION_RATING
raw value repr: 'negative' pd.isna: False
SUBJECT anjana STIM Animals_025_h.jpg EMOTION sad
        stim_cat            stim_id             remarks
450155  negative  Animals_025_h.jpg  sad_EMOTION_RATING
450156  negative  Animals_025_h.jpg  sad_EMOTION_RATING
450157  negative  Animals_025_h.jpg  sad_EMOTION_RATING
450158  negative  Animals_025_h.jpg  sad_EMOTION_RATING
450159  negative  Animals_025_h.jpg  sad_EMOTION_RATING
raw value repr: 'negative' pd.isna: False
SUBJECT anjana STIM Animals_025_h.jpg EMOTION happy
        stim_cat            stim_id               remarks
452285

C:\Users\Deepshik\AppData\Local\Temp\ipykernel_1720\8082256.py:22: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))


SUBJECT krithika STIM Animals_016_h.jpg EMOTION anger
       stim_cat            stim_id               remarks
50095  negative  Animals_016_h.jpg  anger_EMOTION_RATING
50096  negative  Animals_016_h.jpg  anger_EMOTION_RATING
50097  negative  Animals_016_h.jpg  anger_EMOTION_RATING
50098  negative  Animals_016_h.jpg  anger_EMOTION_RATING
50099  negative  Animals_016_h.jpg  anger_EMOTION_RATING
raw value repr: 'negative' pd.isna: False
SUBJECT krithika STIM Animals_016_h.jpg EMOTION sad
       stim_cat            stim_id             remarks
52680  negative  Animals_016_h.jpg  sad_EMOTION_RATING
52681  negative  Animals_016_h.jpg  sad_EMOTION_RATING
52682  negative  Animals_016_h.jpg  sad_EMOTION_RATING
52683  negative  Animals_016_h.jpg  sad_EMOTION_RATING
52684  negative  Animals_016_h.jpg  sad_EMOTION_RATING
raw value repr: 'negative' pd.isna: False
SUBJECT krithika STIM Animals_016_h.jpg EMOTION happy
       stim_cat            stim_id               remarks
56578  negative  Animals_01

In [20]:
import os
import numpy as np
from collections import defaultdict
from pygazeanalyser.detectors import fixation_detection
from utils.common import extract_emotion_rating_segments, convert_to_pygaze_compatible_format

# emotion ratings grouped by stim_cat
def compute_fixation_metrics_collapsed_this_strikes_back(base_dir, save_dir="fixation_metrics_collapsed_2"):
    """
    Compute fixation metrics for emotion rating screens,
    averaged across stimuli within each emotion category.
    Produces both subject-level and group-level summaries.
    """

    subjects = os.listdir(base_dir)
    subjects = [os.path.join(base_dir, s) for s in subjects]

    subject_summaries = []

    for s in subjects:
        subject_id = os.path.splitext(os.path.basename(s))[0]
        df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))

        # --- CLEAN COLUMN NAMES AND STIM_CAT ---
        df.columns = df.columns.str.strip()
        df['stim_cat'] = df['stim_cat'].astype(str).str.strip().replace({'nan': np.nan})

        # Extract rating segments
        rating_segs = extract_emotion_rating_segments(df)

        # Container: store metrics per stimulus per category
        category_metrics = defaultdict(list)

        for stim_id, emotions_dict in rating_segs.items():
            for emotion, seg_df in emotions_dict.items():

                # Assign category
                cat = seg_df['stim_cat'].iloc[0]
                if pd.isna(cat) or str(cat).strip() == "":
                    fallback = df.loc[df['stim_id'] == stim_id, 'stim_cat']
                    if not fallback.empty:
                        cat = (
                            fallback.ffill()
                            .bfill()
                            .dropna()
                            .astype(str)
                            .str.strip()
                            .iloc[0]
                            if fallback.dropna().any()
                            else "UNKNOWN"
                        )
                    else:
                        cat = "UNKNOWN"
                else:
                    cat = str(cat).strip()

                emotion_category = cat

                # --- Calculate fixations ---
                x, y, t = convert_to_pygaze_compatible_format(seg_df)
                _, Efix = fixation_detection(x, y, t)

                durations = [f[2] for f in Efix] if Efix else []

                if durations:
                    # per stimulus metrics
                    fixation_count = len(durations)
                    total_duration = np.sum(durations)
                    mean_duration = np.mean(durations)

                    category_metrics[emotion_category].append({
                        'fixation_count': fixation_count,
                        'total_duration': total_duration,
                        'mean_duration': mean_duration
                    })

        # --- Compute subject-level averages per category ---
        for category, metrics_list in category_metrics.items():
            # convert list of dicts to arrays
            fixation_counts = np.array([m['fixation_count'] for m in metrics_list])
            total_durations = np.array([m['total_duration'] for m in metrics_list])
            mean_durations = np.array([m['mean_duration'] for m in metrics_list])

            subject_summaries.append({
                'subject_id': subject_id,
                'emotion_category': category,
                'fixation_count': fixation_counts.mean(),   # mean across stimuli
                'mean_duration': mean_durations.mean(),    # already mean per fixation
                'total_duration': total_durations.mean()   # mean across stimuli
            })

    # ---- Convert to DataFrame ----
    subject_df = pd.DataFrame(subject_summaries)

    # ---- Group-level summaries ----
    group_df = (
        subject_df.groupby("emotion_category")
        .agg(
            mean_fixation_count=("fixation_count", "mean"),
            sd_fixation_count=("fixation_count", "std"),
            mean_fixation_duration=("mean_duration", "mean"),
            sd_fixation_duration=("mean_duration", "std"),
            mean_total_duration=("total_duration", "mean"),
            sd_total_duration=("total_duration", "std"),
        )
        .reset_index()
    )

    # ---- Save results ----
    os.makedirs(save_dir, exist_ok=True)
    subject_path = os.path.join(save_dir, "subject_level_fixation_metrics_collapsed.csv")
    group_path = os.path.join(save_dir, "group_level_fixation_metrics_collapsed.csv")

    subject_df.to_csv(subject_path, index=False)
    group_df.to_csv(group_path, index=False)

    print(f"Saved subject-level metrics to: {subject_path}")
    print(f"Saved group-level metrics to: {group_path}")

    return subject_df, group_df


subject_df, group_df = compute_fixation_metrics_collapsed_this_strikes_back(base_dir)

C:\Users\Deepshik\AppData\Local\Temp\ipykernel_1720\25813404.py:22: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))


Saved subject-level metrics to: fixation_metrics_collapsed_2\subject_level_fixation_metrics_collapsed.csv
Saved group-level metrics to: fixation_metrics_collapsed_2\group_level_fixation_metrics_collapsed.csv


In [21]:
subject_df

,subject_id,emotion_category,fixation_count,mean_duration,total_duration
0,anjana,negative,6.28,203.338979,1243.57376
1,anjana,neutral,3.62,226.884538,704.71872
2,anjana,positive,5.96,192.799673,1035.09020
3,ankita,negative,8.40,213.109716,1788.27898
4,ankita,neutral,8.32,228.731551,1887.77916
5,ankita,positive,7.20,217.463290,1593.27402
6,deepali,negative,7.54,218.850231,1555.76032
7,deepali,neutral,5.86,235.190427,1199.15678
8,deepali,positive,6.30,203.683872,1210.34328
9,jini,negative,10.58,195.504467,2003.25760


In [22]:
group_df

,emotion_category,mean_fixation_count,sd_fixation_count,mean_fixation_duration,sd_fixation_duration,mean_total_duration,sd_total_duration
0,negative,8.722,3.811963,209.937243,18.899703,1724.946372,691.603498
1,neutral,5.954,1.613775,218.408119,23.627539,1174.059314,354.664409
2,positive,7.062,2.163915,202.055883,17.175595,1362.451240,422.357447


---

In [30]:
def compute_fixation_metrics_per_rating_screen(base_dir, save_dir="fixation_metrics_per_rating_screen"):
    """
    Compute fixation metrics per emotion rating screen, grouped by stimulus category.
    Each row in the subject-level CSV corresponds to one subject × stimulus_category × emotion_rating.
    Produces both subject-level and group-level summaries.
    """

    subjects = os.listdir(base_dir)
    subjects = [os.path.join(base_dir, s) for s in subjects]

    subject_summaries = []

    for s in subjects:
        subject_id = os.path.splitext(os.path.basename(s))[0]
        df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))

        # --- Clean columns ---
        df.columns = df.columns.str.strip()
        df['stim_cat'] = df['stim_cat'].astype(str).str.strip().replace({'nan': np.nan})

        # Extract rating segments
        rating_segs = extract_emotion_rating_segments(df)

        # Container: store metrics per stimulus category × emotion rating
        subj_cat_emotion_metrics = defaultdict(lambda: defaultdict(list))

        for stim_id, emotions_dict in rating_segs.items():
            for emotion_rating, seg_df in emotions_dict.items():

                # Assign stimulus category
                stim_cat = seg_df['stim_cat'].iloc[0]
                if pd.isna(stim_cat) or str(stim_cat).strip() == "":
                    fallback = df.loc[df['stim_id'] == stim_id, 'stim_cat']
                    if not fallback.empty and fallback.dropna().any():
                        stim_cat = fallback.ffill().bfill().dropna().iloc[0]
                    else:
                        stim_cat = "UNKNOWN"
                stim_cat = str(stim_cat).strip()

                # --- Calculate fixations for this segment ---
                x, y, t = convert_to_pygaze_compatible_format(seg_df)
                _, Efix = fixation_detection(x, y, t)

                durations = [f[2] for f in Efix] if Efix else []

                if durations:
                    subj_cat_emotion_metrics[stim_cat][emotion_rating].append({
                        'fixation_count': len(durations),
                        'total_duration': np.sum(durations),
                        'mean_duration': np.mean(durations)
                    })

        # --- Aggregate metrics per stimulus category × emotion rating ---
        for stim_cat, emotion_dict in subj_cat_emotion_metrics.items():
            for emotion_rating, metrics_list in emotion_dict.items():
                fixation_counts = np.array([m['fixation_count'] for m in metrics_list])
                total_durations = np.array([m['total_duration'] for m in metrics_list])
                mean_durations = np.array([m['mean_duration'] for m in metrics_list])

                subject_summaries.append({
                    'subject_id': subject_id,
                    'stimulus_category': stim_cat,
                    'emotion_rating': emotion_rating,
                    'mean_fixation_count': fixation_counts.mean(),
                    'sd_fixation_count': fixation_counts.std(ddof=0),
                    'mean_fixation_duration': mean_durations.mean(),
                    'sd_fixation_duration': mean_durations.std(ddof=0),
                    'mean_total_duration': total_durations.mean(),
                    'sd_total_duration': total_durations.std(ddof=0)
                })

    # ---- Convert to DataFrame ----
    subject_df = pd.DataFrame(subject_summaries)

    # ---- Group-level summaries ----
    group_df = (
        subject_df.groupby(['stimulus_category', 'emotion_rating'])
        .agg(
            mean_fixation_count=('mean_fixation_count', 'mean'),
            sd_fixation_count=('mean_fixation_count', 'std'),
            mean_fixation_duration=('mean_fixation_duration', 'mean'),
            sd_fixation_duration=('mean_fixation_duration', 'std'),
            mean_total_duration=('mean_total_duration', 'mean'),
            sd_total_duration=('mean_total_duration', 'std')
        )
        .reset_index()
    )

    # ---- Save CSVs ----
    os.makedirs(save_dir, exist_ok=True)
    subject_path = os.path.join(save_dir, "subject_level_fixation_metrics.csv")
    group_path = os.path.join(save_dir, "group_level_fixation_metrics.csv")

    subject_df.to_csv(subject_path, index=False)
    group_df.to_csv(group_path, index=False)

    print(f"Saved subject-level metrics to: {subject_path}")
    print(f"Saved group-level metrics to: {group_path}")

    return subject_df, group_df


subject_df, group_df = compute_fixation_metrics_per_rating_screen(base_dir)

C:\Users\Deepshik\AppData\Local\Temp\ipykernel_1720\3187514315.py:15: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))


Saved subject-level metrics to: fixation_metrics_per_rating_screen\subject_level_fixation_metrics.csv
Saved group-level metrics to: fixation_metrics_per_rating_screen\group_level_fixation_metrics.csv


In [33]:
group_df

,stimulus_category,emotion_rating,mean_fixation_count,sd_fixation_count,mean_fixation_duration,sd_fixation_duration,mean_total_duration,sd_total_duration
0,negative,anger,8.20,3.876711,220.691525,25.908086,1733.58380,905.925065
1,negative,disgust,9.21,3.664378,213.623969,36.626919,1872.77890,661.973997
2,negative,fear,8.16,3.416691,205.409271,18.069098,1555.99238,633.689158
3,negative,happy,8.74,5.461013,202.607811,27.261600,1686.00301,946.202560
4,negative,sad,9.30,4.140317,207.353637,39.725458,1776.37377,619.650234
5,neutral,anger,5.31,1.700621,213.775032,26.632804,1054.66550,399.359587
6,neutral,disgust,5.68,1.621933,218.537513,53.003824,1111.51563,336.966337
7,neutral,fear,5.91,1.722047,206.909002,21.975006,1116.28997,368.470639
8,neutral,happy,6.50,2.775688,229.147998,45.474572,1312.62459,570.986870
9,neutral,sad,6.37,1.990003,223.671049,34.338265,1275.20088,383.073806


In [34]:
subject_df

,subject_id,stimulus_category,emotion_rating,mean_fixation_count,sd_fixation_count,mean_fixation_duration,sd_fixation_duration,mean_total_duration,sd_total_duration
0,anjana,negative,disgust,5.8,3.867816,182.505756,111.820010,1144.0043,856.790201
1,anjana,negative,sad,5.3,2.865310,253.870761,55.647748,1213.0913,499.401872
2,anjana,negative,happy,6.9,2.736786,199.591596,34.148457,1360.0898,588.138093
3,anjana,negative,anger,6.3,3.769615,210.744844,46.535081,1313.0101,702.400687
4,anjana,negative,fear,7.1,2.773085,169.981940,29.075631,1187.6733,442.087997
...,...,...,...,...,...,...,...,...,...
145,vaishnavi,neutral,disgust,8.7,7.100000,176.121802,36.276703,1585.3478,1590.659282
146,vaishnavi,neutral,fear,7.7,5.060632,204.739655,96.223735,1290.2600,753.766150
147,vaishnavi,neutral,anger,6.0,3.577709,211.947928,73.627863,1200.1783,764.719877
148,vaishnavi,neutral,happy,13.6,9.318798,337.654370,314.071814,2653.5230,1569.391790
